In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**In this chapter**

- You learn how to model a network using a new, abstract data structure: graphs.                     
- You learn breadth-first search, an algorithm you can run on graphs to answer questions like, “What’s the shortest path to go to X?”                     
- You learn about directed versus undirected graphs.                     
- You learn topological sort, a different kind of sorting algorithm that exposes dependencies between nodes.

> **在本章中**
>
> - 你将学习如何使用一种新的、抽象的数据结构为网络建模：图。                    
> - 你学习广度优先搜索，这是一种你可以在图上运行的算法，用来回答诸如 "去X的最短路径是什么 "这样的问题。                    
> - 你要学习有向图和无向图的知识。                    
> - 你学习拓扑排序，这是一种不同的排序算法，暴露节点之间的依赖关系。

This chapter introduces graphs. First, I’ll talk about what graphs are (they don’t involve an X or Y axis). Then I’ll show you your first graph algorithm. It’s called *breadth-first search* (BFS).

Breadth-first search allows you to find the shortest distance between two things. But shortest distance can mean a lot of things! You can use breadth-first search to      

- Write a checkers AI that calculates the fewest moves to victory                     
- Write a spell checker (fewest edits from your misspelling to a real word—for example, READED -> READER is one edit)                     
- Find the doctor closest to you in your network                     

Graph algorithms are some of the most useful algorithms I know. Make sure you read the next few chapters carefully—these are algorithms you’ll be able to apply again and again.

> 本章介绍了图形。首先，我将谈谈什么是图形（它们不涉及X或Y轴）。然后，我将向你展示你的第一个图形算法。它叫做*宽度优先搜索*（BFS）。
>
> 广度优先搜索允许你找到两个事物之间的最短距离。但最短的距离可以意味着很多东西！你可以使用广度优先搜索。你可以使用广度优先搜索来      
>
> - 编写一个跳棋人工智能，计算出通往胜利的最少的棋步                     
> - 编写一个拼写检查器（从你的错误拼写到一个真正的单词的最少的编辑--例如，READED->READER是一个编辑）                     
> - 在你的网络中找到离你最近的医生                     
>
> 图谱算法是我所知道的最有用的一些算法。请确保你仔细阅读接下来的几章--这些是你能够反复应用的算法。

## Introduction to graphs

Suppose you’re in San Francisco, and you want to go from Twin Peaks to the Golden Gate Bridge. You want to get there by bus, with the minimum number of transfers. Here are your options.

> 假设你在旧金山，你想从双峰山到金门大桥。你想乘坐公交车去那里，并尽量减少转车的次数。以下是你的选择。

<img src="../pic/option roads.png">

What’s your algorithm to find the path with the fewest steps?

Well, can you get there in one step? Here are all the places you can get to in one step.

<img src="../pic/one step.png">

The bridge isn’t highlighted; you can’t get there in one step. Can you get there in two steps?

<img src="../pic/two steps.png">

Again, the bridge isn’t there, so you can’t get to the bridge in two steps. What about three steps?

<img src="../pic/three steps.png">

Aha! Now the Golden Gate Bridge shows up. So it takes three steps to get from Twin Peaks to the bridge using this route. 

<img src="../pic/to golden gate bridge.png">

There are other routes that will get you to the bridge too, but they’re longer (four steps). The algorithm found that the shortest route to the bridge is three steps long. This type of problem is called a *shortest-path problem*. You’re always trying to find the shortest something. It could be the shortest route to your friend’s house. It could be the smallest number of moves to checkmate in a game of chess. The algorithm to solve a shortest-path problem is called *breadth-first search*.

> 还有其他路线也能让你到桥上，但它们更长（四步）。算法发现，到桥的最短路线是三步长。这种类型的问题被称为*最短路径问题*。你总是试图找到最短的东西。它可能是去你朋友家的最短路线。它可能是国际象棋游戏中的最小的将棋步数。解决最短路径问题的算法被称为*广度优先搜索*。

To figure out how to get from Twin Peaks to the Golden Gate Bridge, there are two steps:

**1**. Model the problem as a graph.                              

**2**. Solve the problem using breadth-first search.

Next I’ll cover what graphs are. Then I’ll go into breadth-first search in more detail.

## what is a graph?

A graph models a set of connections. For example, suppose you and your friends are playing poker, and you want to model who owes whom money. Here’s how you could say, “Alex owes Rama money.”

> 图为一组连接的模型。例如，假设你和你的朋友在玩扑克牌，你想模拟谁欠谁的钱。你可以这样说："Alex欠Rama的钱"。

<img src="../pic/alex to rama.png">

The full graph could look something like this.

<img src="../pic/graph of people.png">
Graph of people who owe other people poker money

Alex owes Rama money, Tom owes Adit money, and so on. Each graph is made up of *nodes* and *edges*.

> Alex欠Rama的钱，Tom欠Adit的钱，以此类推。每个图都是由*节点*和*边*组成的。

<img src="../pic/node and edge.png">

That’s all there is to it! Graphs are made up of nodes and edges. A node can be directly connected to many other nodes. Those nodes are called its *neighbors*. In this graph, Rama is Alex’s neighbor. Adit isn’t Alex’s neighbor, because they aren’t directly connected. But Adit is         Rama’s and Tom’s neighbor.

Graphs are a way to model how different things are connected to one another. Now let’s see breadth-first search in action.

> 这就是它的全部内容! 图是由节点和边线组成的。一个节点可以直接连接到许多其他节点。这些节点被称为它的*邻居*。在这个图中，Rama是Alex的邻居。Adit不是Alex的邻居，因为他们没有直接联系。但阿迪特是拉玛和汤姆的邻居。
>
> 图是一种模拟不同事物之间相互联系的方式。现在让我们看看广度优先搜索的操作。

### Breadth-first search

We looked at a search algorithm in chapter 1: binary search. Breadth-first search is a different kind of search algorithm: one that runs on graphs. It can help answer two types of questions:      

- Question type 1: Is there a path from node A to node B?                     
- Question type 2: What is the shortest path from node A to node B?                     

You already saw breadth-first search once, when you calculated the shortest route from Twin Peaks to the Golden Gate Bridge. That was a question of type 2: “What is the shortest path?” Now let’s look at the algorithm in more detail. You’ll ask a question of type 1: “Is there a path?”

> 我们在第一章看了一种搜索算法：二进制搜索。广度优先搜索是一种不同的搜索算法：一种在图上运行的算法。它可以帮助回答两种类型的问题。     
>
> - 问题类型1：是否有一条从节点A到节点B的路径？                    
> - 问题类型2：从节点A到节点B的最短路径是什么？                    
>
> 你已经看过一次广度优先搜索，当时你计算了从双峰山到金门大桥的最短路线。那是一个类型2的问题："什么是最短路径？" 现在让我们更详细地看一下这个算法。你会问一个类型1的问题："是否有一条路径？"

Suppose you’re the proud owner of a mango farm. You’re looking for a mango seller who can sell your mangoes. Are you connected to a mango seller on Facebook? Well, you can search through your friends.

> 假设你是一个芒果农场的自豪主人。你正在寻找一个可以出售你的芒果的芒果卖家。你与Facebook上的芒果卖家有联系吗？那么，你可以通过你的朋友搜索。

<img src="../pic/friends.png">

This search is pretty straightforward. First, make a list of friends to search.

<img src="../pic/list of friends.png">

Now, go to each person in the list and check whether that person sells mangoes.

<img src="../pic/check friends.png">

Suppose none of your friends are mango sellers. Now you have to search through your friends’ friends.

<img src="../pic/friends to friends.png">

Each time you search for someone from the list, add all of their friends to the list.

<img src="../pic/add friends.png">

This way, you not only search your friends, but you search their friends, too. Remember, the goal is to find one mango seller in your network. So if Alice isn’t a mango seller, you add her friends to the list, too. That means you’ll eventually search her friends—and then their friends, and so on. With this algorithm, you’ll search your entire network until you come across a mango seller. This algorithm is breadth-first search.

> 这样一来，你不仅可以搜索你的朋友，而且还可以搜索他们的朋友。记住，目标是在你的网络中找到一个芒果卖家。因此，如果爱丽丝不是芒果卖家，你也把她的朋友加入到列表中。这意味着你最终会搜索她的朋友，然后是他们的朋友，以此类推。通过这种算法，你会搜索你的整个网络，直到你遇到一个芒果卖家。这种算法是广度优先的搜索。

### Finding the shortest path

As a recap, these are the two questions that breadth-first search can answer for you:

- Question type 1: Is there a path from node A to node B? (Is there a mango seller in your network?)                     
- Question type 2: What is the shortest path from node A to node B? (Who is the closest mango seller?)                     

You saw how to answer question 1; now let’s try to answer question 2. Can you find the closest mango seller? For example, your friends are first-degree connections, and their friends are second-degree connections.

> 作为总结，这就是广度优先搜索可以为你回答的两个问题。
>
> - 问题类型1：从节点A到节点B是否有路径？ (你的网络中是否有一个芒果卖家？)                     
> - 问题类型2：从节点A到节点B的最短路径是什么？ (谁是最近的芒果卖家？)                     
>
> 你看到了如何回答问题1；现在让我们试着回答问题2。你能找到最近的芒果卖家吗？例如，你的朋友是一级关系，而他们的朋友是二级关系。

<img src="../pic/friend netwrok.png">

You’d prefer a first-degree connection to a second-degree connection, and a second-degree connection to a third-degree connection, and so on. So you shouldn’t search any second-degree connections before you make sure you don’t have a first-degree connection who is a mango seller. Well, breadth-first search already does this! The way breadth-first search works, the search radiates out from the starting point. So you’ll check first-degree connections before second-degree connections. Pop quiz: who will be checked first, Claire or Anuj? Answer: Claire is a first-degree connection, and Anuj is a second-degree connection. So Claire will be checked before Anuj.

> 你更喜欢一级关系而不是二级关系，二级关系而不是三级关系，以此类推。因此，在你确定你没有一个卖芒果的一级关系之前，你不应该搜索任何二级关系。那么，广度优先搜索已经做到了这一点! 广度优先搜索的工作方式是，搜索从起点向外辐射。所以你会在二级关系之前检查一级关系。流行问答：谁会先被检查，Claire还是Anuj？答案：Claire是一级关系。克莱尔是一级关系，而阿努吉是二级关系。所以Claire会在Anuj之前被检查。

<img src="../pic/degree.png">

Another way to see this is, first-degree connections are added to the search list before second-degree connections.

You just go down the list and check people to see whether each one is a mango seller. The first-degree connections will be searched before the second-degree connections, so you’ll find the mango seller closest to you. Breadth-first search not only finds a path from A to B, it also finds the shortest path.

Notice that this only works if you search people in the same order in which they’re added. That is, if Claire was added to the list before Anuj, Claire needs to be searched before Anuj. What happens if you search Anuj before Claire, and they’re both mango sellers? Well, Anuj is a second-degree contact, and Claire is a first-degree contact. You end up with a mango seller         who isn’t the closest to you in your network. So you need to search people in the order that they’re added. There’s a data structure for this: it’s called *a queue*.

> 另一种方法是，第一级关系在第二级关系之前被添加到搜索列表中。
>
> 你只需顺着列表检查人们，看每一个人是否是芒果卖家。第一层关系会在第二层关系之前被搜索到，所以你会找到离你最近的芒果卖家。广度优先搜索不仅可以找到从A到B的路径，还可以找到最短路径。
>
> 请注意，这只有在你按照人们被添加的顺序进行搜索时才有效。也就是说，如果Claire是在Anuj之前被添加到列表中的，那么Claire需要在Anuj之前被搜索。如果你在Claire之前搜索Anuj，而他们都是芒果卖家，会发生什么？那么，Anuj是二级联系人，而Claire是一级联系人。你最后得到的芒果卖家并不是你网络中最接近的人。所以你需要按照人们被添加的顺序来搜索他们。这方面有一个数据结构：它被称为*队列*。

### Queues

A queue works exactly like it does in real life. Suppose you and your friend are queueing up at the bus stop. If you’re before him in the queue, you get on the bus first. A queue works the same way. Queues are similar to stacks. You can’t access random elements in the queue. Instead, there are two only operations, *enqueue* and *dequeue*.

> 排队的工作原理与现实生活中的一模一样。假设你和你的朋友在公交车站排队。如果你排在他前面，你就先上车。排队的工作方式也是如此。队列类似于堆栈。你不能访问队列中的随机元素。相反，只有两种操作，*enqueue*和*dequeue*。

<img src="../pic/queue.png">

If you enqueue two items to the list, the first item you added will be dequeued before the second item. You can use this for your search list! People who are added to the list first will be dequeued and searched first.      

The queue is called a *FIFO* data structure: First In, First Out. In contrast, a stack is a *LIFO* data structure: Last In, First Out.

> 如果你向列表中的两个项目排队，你添加的第一个项目将在第二个项目之前被取消排队。你可以将此用于你的搜索列表! 先被添加到列表中的人将被取消排队，并先被搜索到。     
>
> 这个队列被称为*FIFO*数据结构。先入先出。与此相反，堆栈是一个*LIFO*数据结构。最后进入，最先出来。

<img src="../pic/FIFO & LIFO.png">

Now that you know how a queue works, let’s implement breadth-first search!

### **Exercises**

Run the breadth-first search algorithm on each of these graphs to find the solution.

**6.1** 

Find the length of the shortest path from start to finish.

<img src="../pic/start to finish.png">

**6.2** 

Find the length of the shortest path from “cab” to “bat”.

<img src="../pic/cab to bat.png">

In [2]:
## 6.1 2
## 6.2 2

## Implementing the graph

First, you need to implement the graph in code. A graph consists of several nodes. And each node is connected to neighboring nodes. How do you express a relationship like “you -> bob”? Luckily, you know a data structure that lets you express relationships: *a hash table*!

Remember, a hash table allows you to map a key to a value. In this case, you want to map a node to all of its neighbors. Here’s how you’d write it in Python:

> 首先，你需要在代码中实现图。一个图由多个节点组成。而每个节点都与相邻的节点相连。你如何表达 "你->鲍勃 "这样的关系？幸运的是，你知道一种数据结构可以让你表达关系。*一个哈希表*!
>
> 记住，哈希表允许你将一个键映射到一个值。在这种情况下，你想把一个节点映射到它所有的邻居。下面是你用 Python 写的方法。

In [3]:
graph = {}
graph["you"] = ["alice", "bob", "claire"]
graph

{'you': ['alice', 'bob', 'claire']}

Notice that “you” is mapped to an array. So graph["you"] will give you an array of all the neighbors of “you”.   

A graph is just a bunch of nodes and edges, so this is all you need to have a graph in Python. What about a bigger graph, like this one?

<img src="../pic/big graph.png">

Here it is as Python code: 

In [5]:
graph = {}
graph["you"] = ["alice", "bob", "claire"]
graph[ "bob"] = ["anuj", "peggy"]
graph["alice"] = ["peggy"]
graph["claire"] = ["thom", "jonny"]
graph["anuj"] = []
graph["peggy"] = []
graph["thom"] = []
graph["jonny"] = []
graph

{'you': ['alice', 'bob', 'claire'],
 'bob': ['anuj', 'peggy'],
 'alice': ['peggy'],
 'claire': ['thom', 'jonny'],
 'anuj': [],
 'peggy': [],
 'thom': [],
 'jonny': []}

Pop quiz: does it matter what order you add the key/value pairs in? Does it matter if you write

```python
graph["claire"] = ["thom", "jonny"]
graph["anuj"] = []
```

instead of

```python
graph["anuj"] = []
graph["claire"] = ["thom", "jonny"]
```



Think back to the previous chapter. Answer: It doesn’t matter. Hash tables have no ordering, so it doesn’t matter what order you add key/value pairs in.      

Anuj, Peggy, Thom, and Jonny don’t have any neighbors. They have arrows pointing to them, but no arrows from them to someone else. This is called a *directed graph*—the relationship is only one way. So Anuj is Bob’s neighbor, but Bob isn’t Anuj’s neighbor. An undirected graph doesn’t have any arrows, and both nodes are each other’s neighbors. For example, both of these graphs are equal.

> 回想一下上一章。答案是。这并不重要。哈希表没有排序，所以你以什么顺序添加键/值对并不重要。     
>
> Anuj, Peggy, Thom, 和Jonny没有任何邻居。他们有指向他们的箭头，但没有从他们到其他人的箭头。这被称为*定向图*-关系只有一个方向。所以Anuj是Bob的邻居，但Bob不是Anuj的邻居。无定向图没有任何箭头，两个节点都是对方的邻居。例如，这两个图都是相等的。

<img src="../pic/equal graph.png">

## Implementing the algorithm

To recap, here’s how the implementation will work.

<img src="../pic/implementation.png">

**Note**

When updating queues, I use the terms *enqueue* and *dequeue*. You’ll also encounter the terms *push* and *pop*. *Push* is almost always the same thing as *enqueue*, and *pop* is almost always the same thing as *dequeue*.

> 在更新队列时，我使用术语*enqueue*和*dequeue*。你还会遇到术语*push*和*pop*。*Push*几乎总是与*enqueue*相同，而*pop*几乎总是与*dequeue*相同。

Make a queue to start. In Python, you use the double-ended queue (deque) function for this:

> 做一个队列来启动。在Python中，你使用双端队列(deque)函数来做这个。

In [7]:
from collections import deque
search_queue = deque()    # creates a new queue
search_queue += graph["you"]    # adds all of your neighbors to the search queue
search_queue

deque(['alice', 'bob', 'claire'])

Remember, graph["you"] will give you a list of all your neighbors, like ["alice", "bob", "claire"]. Those all get added to the search queue.

Let’s see the rest:

One final thing: you still need a person_is_seller function to tell you when someone is a mango seller. Here’s one:

In [10]:
def person_is_seller(name):
    return name[-1] == 'm'

This function checks whether the person’s name ends with the letter *m*. If it does, they’re a mango seller. Kind of a silly way to do it, but it’ll do for this example. Now let’s see the breadth-first search in action.

> 这个函数检查这个人的名字是否以字母*m*结尾。如果是，他们就是一个卖芒果的人。这样做有点傻，但对这个例子来说是可以的。现在让我们看看广度优先搜索的操作。

<img src="../pic/breadth-first search.png">

In [16]:
def search_mango_seller(person):
    search_queue = deque()
    search_queue += graph[person]
    while search_queue:    # while the queue isn't empty
        person = search_queue.popleft()    # grab the first person off the queue
        if person_is_seller(person):    # checks whether the person is a mango seller
            print(person + " is a mango seller!")    # yes, they're mango seller
            return True
        else:
            search_queue += graph[person]    # no, they're not. Add all of this person's friends to search_queue
    return False    # if you reached here, no one in the queue was a mango seller

And so on. The algorithm will keep going until either

- A mango seller is found, or                     
- The queue becomes empty, in which case there is no mango seller.

Alice and Bob share a friend: Peggy. So Peggy will be added to the queue twice: once when you add Alice’s friends, and again when you add Bob’s friends. You’ll end up with two Peggys in the search queue.

> 以此类推。该算法将一直进行下去，直到
>
> - 找到一个芒果卖家，或者                     
> - 队列变空，在这种情况下，就没有芒果卖家了。
>
> 爱丽丝和鲍勃共有一个朋友。佩吉。所以Peggy将被添加到队列中两次：一次是在你添加Alice的朋友时，另一次是在你添加Bob的朋友时。你最终会在搜索队列中出现两个佩吉。

<img src="../pic/two peggy.png">

But you only need to check Peggy once to see whether she’s a mango seller. If you check her twice, you’re doing unnecessary, extra work. So once you search a person, you should mark that person as searched and not search them again.

If you don’t do this, you could also end up in an infinite loop. Suppose the mango seller graph looked like this.

> 但是你只需要检查佩吉一次，看看她是否是卖芒果的。如果你检查她两次，你就是在做不必要的、额外的工作。因此，一旦你搜索了一个人，你就应该把这个人标记为已搜索，不再搜索了。
>
> 如果你不这样做，你也可能最终陷入一个无限循环。假设芒果卖家图看起来像这样。

<img src="../pic/two people.png">

To start, the search queue contains all of your neighbors.

> 首先，搜索队列包含了你所有的邻居。

Now you check Peggy. She isn’t a mango seller, so you add all of her neighbors to the search queue. Next, you check yourself. You’re not a mango seller, so you add all of your neighbors to the search queue. And so on. This will be an infinite loop, because the search queue will keep going from you to Peggy.

> 现在你检查佩吉。她不是卖芒果的，所以你把她所有的邻居都加入到搜索队列中。接下来，你检查自己。你不是卖芒果的，所以你把你所有的邻居都加入到搜索队列中。以此类推。这将是一个无限的循环，因为搜索队列将一直从你到佩吉。

<img src="../pic/infinite loop.png">

Before checking a person, it’s important to make sure they haven’t been checked already. To do that, you’ll keep a list of people you’ve already checked.

Here’s the final code for breadth-first search, taking that into account:

> 在检查一个人之前，重要的是要确保他们还没有被检查过。为了做到这一点，你要保留一个你已经检查过的人的列表。
>
> 下面是考虑到这一点的广度优先搜索的最终代码。

In [19]:
def search_mango_seller(name):
    search_queue = deque()
    search_queue += graph[name]
    searched = []    # this arrary is how you keep track of whic person you've searched
    while search_queue:    # while the queue isn't empty
        person = search_queue.popleft()    # grab the first person off the queue
        if not person in searched:
            if person_is_seller(person):    # checks whether the person is a mango seller
                print(person + " is a mango seller!")    # yes, they're mango seller
                return True
            else:
                search_queue += graph[person]    # no, they're not. Add all of this person's friends to search_queue
                searched.append(person)    # marked this person as searched
    return False    # if you reached here, no one in the queue was a mango seller

In [20]:
search_mango_seller("you")

thom is a mango seller!


True

Try running this code yourself. Maybe try changing the person_is_seller function to something more meaningful, and see if it prints what you expect.

> 试着自己运行这段代码。也许可以试着把person_is_seller函数改成更有意义的东西，看看是否能打印出你所期望的东西。

**Running time**

If you search your entire network for a mango seller, that means you’ll follow each edge (remember, an edge is the arrow or connection from one person to another). So the running time is at least O(number of edges).      

You also keep a queue of every person to search. Adding one person to the queue takes constant time: O(1). Doing this for every person will take O(number of people) total. Breadth-first search takes O(number of people + number of edges), and it’s more commonly written as O(V+E) (V for number of vertices, E for number of edges).

> 如果你在整个网络中搜索芒果卖家，这意味着你要跟踪每条边（记住，边是一个人到另一个人的箭头或连接）。所以运行时间至少是O（边的数量）。     
>
> 你还保留了一个每个人的搜索队列。在队列中增加一个人需要恒定的时间。O(1). 对每个人都这样做，总共需要O(人的数量)。广度优先搜索需要O(人的数量+边的数量)，更常见的写法是O(V+E)（V代表顶点的数量，E代表边的数量）。

### **Exercise**

Here’s a small graph of my morning routine.

> 这是我的晨练小图。

<img src="../pic/morning route.png">

It tells you that I can’t eat breakfast until I’ve brushed my teeth. So “eat breakfast” *depends on* “brush teeth”.      

On the other hand, showering doesn’t depend on brushing my teeth, because I can shower before I brush my teeth. From this graph, you can make a list of the order in which I need to do my morning routine:     

1. Wake up.                     
2. Shower.                     
3. Brush teeth.
4. Eat breakfast.

Note that “shower” can be moved around, so this list is also valid:    

1. Wake up.                     
2. Brush teeth.                     
3. Shower.                     
4. Eat breakfast.

**6.3** 

For these three lists, mark whether each one is valid or invalid.

<img src="../pic/three lists.png">

**6.4** 

Here’s a larger graph. Make a valid list for this graph.

<img src="../pic/larger graph.png">

You could say that this list is sorted, in a way. If task A depends on task B, task A shows up later in the list. This is called a *topological sort*, and it’s a way to make an ordered list out of a graph. Suppose you’re planning a wedding and have a large graph full of tasks to do—and you’re not sure where to start. You could *topologically sort* the graph and get a list of tasks to do, in order.

> 你可以说，这个列表在某种程度上是被排序的。如果任务A依赖于任务B，那么任务A就会在列表的后面显示出来。这被称为*拓扑排序*，它是一种从图中制作有序列表的方法。假设你正在计划一场婚礼，有一个很大的图，里面有很多任务要做，而你不确定从哪里开始。你可以对图进行*拓扑排序*，得到一个按顺序排列的任务列表。

Suppose you have a family tree.

> 假设你有一个家庭树。

<img src="../pic/family tree.png">

This is a graph, because you have nodes (the people) and edges. The edges point to the nodes’ parents. But all the edges go down—it wouldn’t make sense for a family tree to have an edge pointing back up! That would be meaningless—your dad can’t be your grandfather’s dad!

> 这是一个图，因为你有节点（人）和边。边缘指向节点的父母。但是所有的边都是向下的--如果家谱上有一条边指向上面，那就没有意义了！那就没有意义了--你的爸爸不可能是你爷爷的爸爸！"。

This is called a *tree*. A tree is a special type of graph, where no edges ever point back.

> 这被称为*树*。树是一种特殊类型的图，其中没有边缘往回指。

**6.5** 

Which of the following graphs are also trees?

<img src="../pic/which is a tree.png">

## Recap

- Breadth-first search tells you if there’s a path from A to B.                     
- If there’s a path, breadth-first search will find the shortest path.                     
- If you have a problem like “find the shortest X,” try modeling your problem as a graph, and use breadth-first search to solve.                     
- A directed graph has arrows, and the relationship follows the direction of the arrow (rama -> adit means “rama owes adit money”).                     
- Undirected graphs don’t have arrows, and the relationship goes both ways (ross - rachel means “ross dated rachel and rachel dated ross”).                     
- Queues are FIFO (First In, First Out).                     
- Stacks are LIFO (Last In, First Out).                     
- You need to check people in the order they were added to the search list, so the search list needs to be a queue. Otherwise, you won’t get the shortest path.
- Once you check someone, make sure you don’t check them again. Otherwise, you might end up in an infinite loop.

> - 广度优先搜索告诉你是否有一条从A到B的路径。                    
> - 如果有路径，广度优先搜索将找到最短的路径。                    
> - 如果你有一个类似于 "找到最短的X "的问题，试着将你的问题建模为一个图，并使用广度优先搜索来解决。                    
> - 有向图有箭头，关系遵循箭头的方向（rama -> adit意味着 "rama欠adit的钱"）。                    
> - 无向图没有箭头，关系是双向的（ross - rachel表示 "ross与rachel约会，rachel与ross约会"）。                    
> - 队列是FIFO（先入先出）。                    
> - 堆栈是后进先出（LIFO）。                    
> - 你需要按照人们被添加到搜索列表的顺序来检查他们，所以搜索列表需要是一个队列(queue)。否则，你就不会得到最短的路径。
> - 一旦你检查了某人，确保你不会再检查他们。否则，你可能会陷入一个无限的循环。